To do:
- Make first_note and last_note work
- Fix 9/11/sus2/sus4 detection
- Fix chord_changes
- Make melodies work

In [444]:
import pretty_midi as pm
import matplotlib.pyplot as plt
# Env variables
chrom_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B'] # A list of all the notes/pitch classes with
                                                                                # indices corresponding to 
                                                                                # MIDI note values mod 12

offsets = { # A list of chord intervals with their corresponding MIDI note value offset
    '1': 0, 
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11,
    '8': 12,
    '9': 14,
    '10': 16,
    '11': 17,
    '12': 19,
    '13': 21
}

In [445]:
# Returns the first note by time in a list of notes
def first_note(notes):
    if notes == []:
        return None
    f_note = notes[0]
    for i in range(1, len(notes)):
        if notes[i].start < f_note.start:
            f_note = notes[i].start
    return f_note

# Returns the last note by time in a list of notes
def last_note(notes):
    if notes == []:
        return None
    l_note = notes[0]
    for i in range(1, len(notes)):
        if notes[i].end > l_note.end:
            l_note = notes[i].end
    return l_note

# Returns a list of all the non-drum notes in a song regardless of instrument
def consolidate_notes(song):
    notes = []
    for instrument in song.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append(note)
    return notes

# Returns a note name based on its MIDI note number
def get_note(note_n):
    return chrom_notes[note_n % 12]

# Returns the note corresponding to a particular degree in a scale defined by the root note
def parse_chord(root, number_string):
    note_num = chrom_notes.index(root)
    out = ""
    num = ""
    scale_num = 0
    parentheses = False
    for char in number_string:
        if char == '(':
            parentheses = True
        if char == 'b':
            scale_num -= 1
        if char == '#':
            scale_num += 1
        if char >= '0' and char <= '9':
            num += char
    scale_num += offsets.get(num)
    if (parentheses):
        out = "("
    out += str(get_note(note_num + scale_num))
    if (parentheses):
        out += ")"
    return out

# Outputs the master chord list (dict version)
def generate_chord_list(filepath = "chords without names.txt"):
    chord_list = []
    for note in chrom_notes:
        f = open(filepath)
        lines = f.readlines()
        for line in lines:
            parts = line.split()
            chord_name = ''
            note_list = []
            for i in range(len(parts)):
                part = parts[i]
                if i == 0:
                    chord_name = part.replace('_', note, 1)
                elif part[0] == 'b' or part[0] == '#' or \
                   (part[0] >= '0' and part[0] <= '9') or \
                   part[0] == '(':
                    note_list.append(parse_chord(note, part))
                else: continue
            chord_list.append([chord_name, note_list])
    return chord_list

# Gets the chords in a song
def get_chords(notes, 
               offset = 0.01):
    start_times = []
    for note in notes:
        if not (note.start in start_times):
            start_times.append(note.start)
    chords = []
    for time in start_times:
        playing_notes = []
        actual = time + offset # Offset is a parameter shifting the time selected for to allow chords to be picked up
        for note in notes:
            if note.start < actual and note.end >= time:
                song.instruments[0].notes
                playing_notes.append(note)
        chords.append(playing_notes)
    return chords

# Generates note scores for the piece
def get_note_scores(notes, 
                    octave_multiplier_on = False,
                    end_multiplier_on = False,
                    printVals = False):
    note_scores_octave_agn = []
    note_scores_octave_agn_dict = dict()
    last_end = last_note(notes).end
    first_start = first_note(notes).start
    overall_dur = last_end - first_start
    for i in range(0, 12):
        note_scores_octave_agn.append(0) # Create bins for each note
    for note in notes:
        duration = note.end - note.start
        score = duration * note.velocity / 127
        octave_multiplier = 1
        end_multiplier = 1
        if octave_multiplier_on: # Reduce the score of the note the higher up in pitch it is
            octave_multiplier = max(0, 1 - (max(0, (round(note.pitch / 12) - 2) / 8.0)))
        if end_multiplier_on: # Reduce the score of the note the farther away it is from the last note
            end_multiplier = note.end / last_end
        score *= octave_multiplier
        score *= end_multiplier
        note_scores_octave_agn[note.pitch % 12] += score # Add the note scores by pitch class
    for i in range(0, 12):
        if note_scores_octave_agn[i] != 0:
            note_scores_octave_agn_dict[i] = note_scores_octave_agn[i]
    if printVals:
        note_scores_octave_agn_sorted = sorted(note_scores_octave_agn_dict.items(), key=lambda x:x[1], reverse = True)
        print(inst.name)
        for key in note_scores_octave_agn_dict.keys():
            print("Note %s has a score of %f" % (get_note(key), note_scores_octave_agn_dict[key]))
        print("The most common note is %s" % get_note(note_scores_octave_agn_sorted[0][0]))
        print("------")
        print()
    else:
        return note_scores_octave_agn_dict, overall_dur, last_end, first_start
    
# Generates chord scores based on note scores
def get_chord_scores(chord_list, 
                     note_scores_octave_agn_dict, 
                     overall_dur,
                     parentheses_multiplier = 1,
                     min_note_threshold = 0.1, 
                     missing_deweight = 0.5, 
                     root_note_multiplier = 1,
                     printVals = False):
    chord_scores_dict = {}
    for chord_tuple in chord_list:
        chord_name = chord_tuple[0]
        chord_notes = chord_tuple[1]
        chord_score = 0.0
        for i in range(0, len(chord_notes)):
            note = chord_notes[i]
            multiplier = 1 # A multiplier for the note score when calculating chord matchups
            actual_note = note
            if note[0] == '(':
                multiplier = parentheses_multiplier
                actual_note = note[1 : (len(note) - 1)]
            if i == 0: # If the note is the root note, weight that pitch specifically
                multiplier *= root_note_multiplier
            note_val = chrom_notes.index(actual_note)
            note_score = note_scores_octave_agn_dict.get(note_val, 0) # Grab the actual note score
            if note_score <= min_note_threshold: # Deweight chords with missing notes
                note_score = -1 * missing_deweight
            chord_score += note_score * multiplier # Multiply by the multiplier and sum to the chord score
        if chord_score > 0.0:
            chord_scores_dict[chord_name] = chord_score
    chord_scores_dict_sorted = sorted(chord_scores_dict.items(), key=lambda x:x[1], reverse = True) # Sort the chords
    if printVals:
        print("The 10 highest-scoring chords are:")
        for i in range(10):
            print("%d: %s with a score of %f" % ((i + 1), chord_scores_dict_sorted[i][0], chord_scores_dict_sorted[i][1]))
    else:
        return chord_scores_dict_sorted

# Makes a list of all the chords in a song
def calculate_song_chords(song):
    all_chords = generate_chord_list()
    chord_list = []
    for chord in get_chords(song.instruments[0].notes):
        note_dict, overall_dur, last_end, first_start = get_note_scores(chord)
        chord = get_chord_scores(all_chords, note_dict, last_end)[:1][0][0]
        if chord != "":
            chord_list.append(chord)
    return chord_list

# Returns n-grams of the items of a list (used in this case to make chord-grams)
def n_grams(my_list, n):
    items = []
    for i in range(0, len(my_list) - n):
        n_gram = []
        for j in range(i, i + n):
            n_gram.append(my_list[j])
        items.append(n_gram)
    return items

# Returns the number of chord changes per second on average of a song (Not working)
# def chord_changes(chord_list, song):
#    notes = song.instruments[0].notes
#    duration = last_note(notes).end - first_start
#    chord_changes_per_time = (len(chord_list) - 1) / duration
#    return chord_changes_per_time

In [446]:
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\weird.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Space Theme.mid")
song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\gen hoshino-Piano.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.midi")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\progression.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Untitled score.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\7-Strings Ensemble Staccato 7 test.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\jazz but without the piano chords.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\blind comp render E-PIANO ONLY_basic_pitch.mid")

In [447]:
all_chords = generate_chord_list()
notes = consolidate_notes(song)
for chord in get_chords(notes):
    note_dict, overall_dur, last_end, first_start = get_note_scores(chord, octave_multiplier_on = True)
    # print(note_dict)
    print(get_chord_scores(all_chords, note_dict, last_end)[:1], end = " @ ")
    print(f"%.2f s" % first_start)

[('Bsus4', 1.5102363463254593)] @ 0.00 s
[('Bm', 0.5558226737532809)] @ 1.26 s
[('Bo7', 0.7543307715223097)] @ 1.89 s
[('AM9', 1.8281808402887139)] @ 2.53 s
[('Balt7', 0.7543307715223095)] @ 3.79 s
[('Gb0', 0.6042271535433066)] @ 4.42 s
[('Dbm9', 1.5734771307086606)] @ 4.96 s
[] @ 6.00 s
[('Bm7', 0.7146291519685039)] @ 6.32 s
[('E7', 0.7543307715223095)] @ 6.95 s
[] @ 7.58 s
[] @ 7.89 s
[('Gbm9', 2.6239538034120744)] @ 8.21 s
[('AM9', 3.658268021391077)] @ 10.11 s
[('Dsus2', 1.0062992964566915)] @ 12.63 s
[('Dbm9', 2.544219026771654)] @ 13.47 s
[('Dbsus4', 2.703937233202101)] @ 15.16 s
[('Asus2', 1.8881891337270311)] @ 17.68 s
[('Gbsus4', 1.1322835589238835)] @ 19.26 s
[('Asus2', 2.2661419211286113)] @ 20.21 s
[('Abm7', 0.7146291519685058)] @ 22.11 s
[('AM9', 2.743224430839893)] @ 22.74 s
[('Abaug7', 0.7146291519685058)] @ 24.63 s
[('Dbm9', 3.3399091055118113)] @ 25.26 s
[] @ 27.47 s
[('AM9', 1.3706590450131222)] @ 27.79 s
[('Gb7sus4', 3.8173231527559066)] @ 28.74 s
[('Gbsus4', 1.13228

In [448]:
# chords = calculate_song_chords(song)
# print(chord_changes(chords, song))